# Hard disks: From Classical to Statistical Mechanics

- Hard disk: two-dimmensional hard spheres

## Molecular Dynamics for Hard Disks
![Hard disks](images/hard_disks_event.gif)

- Event driven molecular dynamics algorithm (Alder and Wainwright, 1957)
    - Wall Collision 
    - Pair Collision
- Because of the negative curvature of the disks $\Rightarrow$ Chaos (bad numerical stability)

In [ ]:
# Above simulation:

import math

def wall_time(pos_a, vel_a, sigma):
    if vel_a > 0.0:
        del_t = (1.0 - sigma - pos_a) / vel_a
    elif vel_a < 0.0:
        del_t = (pos_a - sigma) / abs(vel_a)
    else:
        del_t = float('inf')
    return del_t

def pair_time(pos_a, vel_a, pos_b, vel_b, sigma):
    del_x = [pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]]
    del_x_sq = del_x[0] ** 2 + del_x[1] ** 2
    del_v = [vel_b[0] - vel_a[0], vel_b[1] - vel_a[1]]
    del_v_sq = del_v[0] ** 2 + del_v[1] ** 2
    scal = del_v[0] * del_x[0] + del_v[1] * del_x[1]
    Upsilon = scal ** 2 - del_v_sq * ( del_x_sq - 4.0 * sigma **2)
    if Upsilon > 0.0 and scal < 0.0:
        del_t = - (scal + math.sqrt(Upsilon)) / del_v_sq
    else:
        del_t = float('inf')
    return del_t

# Initial Positions, Velocities and Time
pos = [[0.25, 0.25], [0.75, 0.25], [0.25, 0.75], [0.75, 0.75]]
vel = [[0.21, 0.12], [0.71, 0.18], [-0.23, -0.79], [0.78, 0.1177]]
t = 0.0
# All indexes of pos, or vel
singles = [(i,j) for i in range(4) for j in range(2)]
# All disk combinations
pairs = list(iter.combinations(range(4),2))

# Sigma = radius
sigma = 0.15
n_events = 100
for event in range(n_events):
    wall_times = [wall_time(pos[k][l], vel[k][l], sigma) for k, l  in singles]
    pair_times = [pair_time(pos[k], vel[k], pos[l], vel[l], sigma) for k, l in pairs]
    next_event = min(wall_times + pair_times)
    t += next_event
    for k, l in singles: pos[k][l] += vel[k][l] * next_event 
    if min(wall_times) < min(pair_times):
        collision_disk, direction = singles[wall_times.index(next_event)]
        vel[collision_disk][direction] *= -1.0
    else:
        a, b = pairs[pair_times.index(next_event)]
        del_x = [pos[b][0] - pos[a][0], pos[b][1] - pos[a][1]]
        abs_x = math.sqrt(del_x[0] ** 2 + del_x[1] ** 2)
        e_perp = [c / abs_x for c in del_x]
        del_v = [vel[b][0] - vel[a][0], vel[b][1] - vel[a][1]]
        scal = del_v[0] * e_perp[0] + del_v[1] * e_perp[1]
        for k in range(2): 
            vel[a][k] += e_perp[k] * scal 
            vel[b][k] -= e_perp[k] * scal 
    print('event', event)
    print('time', t)
    print('pos', pos)
    print('vel', vel)

## Statistical mechanics
- Ergodic Hypothesis: over long periods of time, the time spent by a system in some region of the phase space of microstates with the same energy is proportional to the volume of this region, i.e., that all accessible microstates are equiprobable over a long period of time.
- All configurations of hard disks have the same probalibity $\pi(a) = \pi(b)$ (no velocity)
- $\rightarrow$ Direct sampling with Monte Carlo algorithm
- From thermodynamic point of view, no difference between Newtonian classical mechanics and Boltzmann statistical mechanics
- Place disks one after another randomly
- If a placement is illegal $\rightarrow$ Tabula Rasa (start from beginning)


In [ ]:
import random, math, os, pylab

def direct_disks_box(N, sigma):
    condition = False
    while condition == False:
        L = [(random.uniform(sigma, 1.0 - sigma), random.uniform(sigma, 1.0 - sigma))]
        for k in range(1, N):
            a = (random.uniform(sigma, 1.0 - sigma), random.uniform(sigma, 1.0 - sigma))
            min_dist = min(math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) for b in L) 
            if min_dist < 2.0 * sigma: 
                condition = False
                break
            else:
                L.append(a)
                condition = True
    return L

def snapshot(pos, colors):
    pylab.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.10)
    pylab.gcf().set_size_inches(6, 6)
    pylab.axis([0, 1, 0, 1])
    pylab.setp(pylab.gca(), xticks=[0, 1], yticks=[0, 1])
    for (x, y), c in zip(pos, colors):
        circle = pylab.Circle((x, y), radius=sigma, fc=c)
        pylab.gca().add_patch(circle)
    pylab.show()
    pylab.close()

N = 4
colors = ['r', 'b', 'g', 'orange']
sigma = 0.2
n_runs = 8
for run in range(n_runs):
    pos = direct_disks_box(N, sigma)
    snapshot(pos, colors)


## Markov Disks
- From a configuration $\pi$ move one random disk
- If possible $\rightarrow$ new configuration
- If rejected $\rightarrow$ configuration does not change

We must check irreducibility and aperiodicity
- Aperiodicity trivial
- irreducibility: disks have to be small enough to actually freely move around (for example the monte carlo above)

In [ ]:
import random, os, pylab

def snapshot(pos, colors):
    global img
    pylab.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.10)
    pylab.gcf().set_size_inches(6, 6)
    pylab.axis([0, 1, 0, 1])
    pylab.setp(pylab.gca(), xticks=[0, 1], yticks=[0, 1])
    for (x, y), c in zip(pos, colors):
        circle = pylab.Circle((x, y), radius=sigma, fc=c)
        pylab.gca().add_patch(circle)
    pylab.show()
    pylab.close()

L = [[0.25, 0.25], [0.75, 0.25], [0.25, 0.75], [0.75, 0.75]]
sigma = 0.15
sigma_sq = sigma ** 2
delta = 0.1
colors = ['r', 'b', 'g', 'orange']
n_steps = 50
for step in range(n_steps):
    snapshot(L, colors)
    a = random.choice(L)
    b = [a[0] + random.uniform(-delta, delta), a[1] + random.uniform(-delta, delta)]
    # Test for overlaps
    min_dist = min((b[0] - c[0]) ** 2 + (b[1] - c[1]) ** 2 for c in L if c != a) 
    box_cond = min(b[0], b[1]) < sigma or max(b[0], b[1]) > 1.0 - sigma
    if not (box_cond or min_dist < 4.0 * sigma ** 2):
        a[:] = b


- Equiprobability principal
- $\pi(a) = \pi(E(A))$
- For hard disks: $\pi(a) = \pi(b) = \pi(c)$

## Random Sequential Deposition
- Instead of Tabula Rasa rule, the rejected disk is placed again.
- **It's configurations are not equaly probable**

### Discrete hard rod modell
- Positions $0-4$
- Two rods cannot be placed directly next to each other
    - (Valid configuration: 0, 3; Invalid configuration: 1,2)
- With random sequential deposition
    - Depending on the position of the first rod, the second rod has either one or two options.
    $\rightarrow$ different probabilties ($1/4$ or $1/8$) $\neq$ Equiprobability principal
- With tabula rasa:
    - All probabilities are equal ($\pi(a) = \pi(b) = 1/6$)

$\Rightarrow$ applies for hard disks


In [ ]:
import random
configurations = {(0, 3): 'a', (0, 4): 'b', (1, 4): 'c', 
                  (3, 0): 'd', (4, 0): 'e', (4, 1): 'f'}

def random_sequential_descrete(runs = 10000):
    counts = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0}
    for i in range(runs):
        red_rod = random.randint(0, 3)
        if red_rod >= 2: red_rod += 1
        while True:
            blue_rod = random.randint(0, 4)
            if abs(red_rod - blue_rod) > 2: break
        conf = configurations[(red_rod, blue_rod)]  
        counts[conf] += 1
    return counts

def tabula_rasa_descrete(runs = 10000):
    counts = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0}
    for i in range(runs):
        while True:
            red_rod = random.randint(0,4)
            blue_rod = random.randint(0,4)
            if abs(red_rod - blue_rod) > 2:
                break
        conf = configurations[(red_rod, blue_rod)]
        counts[conf] += 1
    return counts

# Compare sequentail and tabula rasa
runs = 10000
seq = random_sequential_descrete(runs)
tab = tabula_rasa_descrete(runs)
print('Conf | Sequential | Tabula Rasa')
for conf in 'abcdef':
    print(conf, seq[conf] / float(runs), tab[conf] / float(runs))

## Hard disks with periodic boundaries
- Periodic boundaries
    - Disks overlap also over the boundary
- New MC simulation with N disks and density $\eta$
- $N \pi \sigma^2 = \eta$
- at $\eta = 30\%$ and $N = 16$ the acceptance ratio is $5 \times 10^{-6}$
- with $N = 16 \Rightarrow 16$ Dimensions of the space
- $\{ \text{volume of box} \} = V$
- $\{ \text{volume of 32-dim space} \} = V^{16} = V^N$
- **Partition Function**: $Z$
- Disk: $(x, y) = x_i$
- $\{ \text{number of legal configurations for } \eta = 0 \} = Z(\eta = 0) = \int_V dx_0 \dots \int_V dx_{N-1} = V^N$
- $\Rightarrow p_{\text{accept}} = 1$
- at $\eta = 0.3, N = 16 \Rightarrow p_{\text{accept}} = 5 \times 10^{-6}$
- $Z(\eta = 0.3) = \int_V dx_0 \dots \int_V dx_{N-1} \pi(x_0, \dots, x_{N-1}) = Z(0) p_{\text{accept}}(\eta = 0.3)$

In [ ]:
import random, math, pylab, os

# Implements periodic boundaries
def dist(x,y):
    d_x = abs(x[0] - y[0]) % 1.0
    d_x = min(d_x, 1.0 - d_x)
    d_y = abs(x[1] - y[1]) % 1.0
    d_y = min(d_y, 1.0 - d_y)
    return  math.sqrt(d_x**2 + d_y**2)

# Creates N disks with radius sigma
def direct_disks(N, sigma):
    n_iter = 0
    condition = False
    while condition == False:
        n_iter += 1
        L = [(random.random(), random.random())]
        for k in range(1, N):
            a = (random.random(), random.random())
            min_dist = min(dist(a, b) for b in L) 
            if min_dist < 2.0 * sigma: 
                condition = False
                break
            else:
                L.append(a)
                condition = True
    return n_iter, L

def snapshot(pos, colors, border_color = 'k'):
    pylab.figure()
    pylab.axis([0, 1, 0, 1])
    #[i.set_linewidth(2) for i in pylab.gca().spines.itervalues()]
    #[i.set_color(border_color) for i in pylab.gca().spines.itervalues()]
    pylab.setp(pylab.gca(), xticks = [0, 1], yticks = [0, 1], aspect = 'equal')
    for (x, y), c in zip(pos, colors):
        circle = pylab.Circle((x, y), radius = sigma, fc = c)
        pylab.gca().add_patch(circle)
    pylab.show()
    pylab.close()

def periodicize(config):
    images = [-1.0, 0.0, 1.0]
    return [(x + dx, y + dy) for (x,y) in config for dx in images for dy in images]

N = 16
eta = 0.28
sigma = math.sqrt(eta / N / math.pi)
n_runs = 6
colors = ['r' for i in range(8 * N)]
for run in range(n_runs):
    iterations, config =  direct_disks(N, sigma)
    print('run',run)
    print(iterations - 1, 'tabula rasa wipe-outs before producing the following configuration')
    config_per = periodicize(config)
    snapshot(config_per, colors, border_color = 'k')

## Hard Disks: Virial Expansions
- Determine $p_{\text{accept}}$ at any $\eta$ of the direct sampling (= Monte Carlo)
- $\eta = \frac{N\pi\sigma^2}{V}$
- $p_{\text{accept}}(\eta) = 1 - \int_0^\eta d\eta_{\text{max}} \pi(\eta_{\text{max}})$
- $Z(\eta) = \int dx_0 \dots dx_{N-1} \pi(x_0, \dots, x_{N-1})$
- $\pi(x_0, \dots, x_{N-1}) = \begin{cases}1 & \text{if there is no overlap} \\ 0 & \text{otherwise} \end{cases}$
- $\Upsilon(x_k, x_l) = \begin{cases}1 & \text{if dist}(x_k,x_l) < 2\sigma \\ 0 & \text{otherwise} \end{cases}$
- $\Rightarrow Z(\eta) = \int dx_0 \dots dx_{N-1} [1-\Upsilon(x_0, x_1)]\dots[1-\Upsilon(x_{N-2},x_{N-1})]$
- $\frac{1}{2} N(N-1)$ factors of $[1-\Upsilon(x_k, x_l)]$. If one factorial is 0 $\rightarrow$ then the contribution of the configuration in the integral vanishes.
- **Partition function is exact, but it cannot be computed analyticaly**
- $\forall \Upsilon(x_i, x_j) = 0 \Rightarrow \forall [ \dots ] = 1 \Rightarrow \int dx_0\dots dx_{N-1} = Z(0) = V^N$
    - 0th order of the expansion
- $\int dx_k dx_l \Upsilon(dx_k, dx_l) = V \int dx_k \Upsilon(x_k, x_l) = V \cdot 4 \pi \sigma^2$
    - 1st order of the expansion $\Rightarrow$ pick one pair
    - $\int dx_k \Upsilon(x_k, x_l)$ is the excluded volume for particle $x_k$
- $\Rightarrow Z(\eta) = V^N - \frac{1}{2} N(N-1) \cdot V^{N-1} \cdot 4 \pi \sigma^2$
    - $V^{N-2}$ for intergrals other then $\int dx_k dx_l$
- $Z(\eta) = V^N (1 - 4\pi\sigma^2 \frac{N(N-1)}{2V} + \dots) \simeq V^N \exp[-2(N-1)\eta]$
- $\Rightarrow \log Z(\eta) = N \log V - 2(N-1)\eta + \dots$
- $\frac{\partial \log Z(\eta)}{\partial V} = \frac{N}{V} + 2N(N-1)\pi\sigma^2 \frac{1}{V^2} + \dots$
- $\Rightarrow \frac{N}{V} \frac{\partial \log Z(\eta)}{\partial V} = 1 + 2(N-1)\pi\sigma^2 \frac{1}{V} + \dots$
- $B = 2(N-1)\pi\sigma^2$


- Virial expansion: $\frac{p}{k_B T} = \rho + B_2(T)\rho^2 + B_3(T)\rho^3 + \dots$
- $\rho = \frac{N}{V}$
- $\beta = (k_B T)^{-1}$
- $\frac{\beta p}{\rho} = 1 + B_2(T) \rho^1 + B_3(T) \rho^2 + \dots$



- Valid only upto some certain $\eta$
- Does not account phase transition
    

In [ ]:
import random, math, pylab

def dist(x, y):
    d_x = abs(x[0] - y[0]) % 1.0
    d_x = min(d_x, 1.0 - d_x)
    d_y = abs(x[1] - y[1]) % 1.0
    d_y = min(d_y, 1.0 - d_y)
    return  math.sqrt(d_x**2 + d_y**2)
    
N = 16
n_confs = 10 ** 5
pairs = [(i, j) for i in range(N - 1) for j in range(i + 1, N)]
eta_max_list = []
for conf in range(n_confs):
    L = [(random.random(), random.random()) for k in range(N)]
    sigma_max = min(dist(L[i], L[j]) for i, j in pairs) / 2.0
    eta_max = N * math.pi * sigma_max ** 2
    eta_max_list.append(eta_max)

# Begin of graphics output
pylab.figure()
n, bins, patches = pylab.hist(eta_max_list, 100, histtype='step', cumulative=-1, 
                   log=True, normed=True, label="numerical evaluation of p_accept")
explaw = [math.exp( - 2.0 * (N - 1) * eta) for eta in bins]
pylab.plot(bins, explaw, 'r--', linewidth=1.5, label="1st order virial expansion")
pylab.xlabel('density eta')
pylab.ylabel('p_accept(eta)')
pylab.legend()
pylab.show()